In [ ]:
# if this doens't work, download directly from 
# https://storage.cloud.google.com/doc-extraction/exploration_data.zip
# and move the .zip file to this folder (/doc-extraction/src)
!dvc pull
# if this doesn't work, launch a terminal and type it again
!unzip exploration_data.zip

In [2]:
from PIL import ImageFilter, Image
import pytesseract
import IPython.display 
from  pdf2image import convert_from_path
from os import listdir
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [3]:
# PARAMETERS OF pdf2image.convert_from_path()
path = 'exploration_data/'
dpi = 450 # TODO(leommiranda): test lower dpi values with the ocr algorithm
first_page = 1
last_page = 2 # value None means that the last page will be the last
thread_count = 1
size = None


#PARAMETERS OF pytesseract.image_to_string(), except for --lang parameter
"""
--psm
    0 = Orientation and script detection (OSD) only.
    1 = Automatic page segmentation with OSD.
    2 = Automatic page segmentation, but no OSD, or OCR. (not implemented)
    3 = Fully automatic page segmentation, but no OSD. (Default)
    4 = Assume a single column of text of variable sizes.
    5 = Assume a single uniform block of vertically aligned text.
    6 = Assume a single uniform block of text.
    7 = Treat the image as a single text line.
    8 = Treat the image as a single word.
    9 = Treat the image as a single word in a circle.
    10 = Treat the image as a single character.
    11 = Sparse text. Find as much text as possible in no particular order.
    12 = Sparse text with OSD.
    13 = Raw line. Treat the image as a single text line,
         bypassing hacks that are Tesseract-specific.

--oem
    0 = Original Tesseract only.
    1 = Neural nets LSTM only.
    2 = Tesseract + LSTM.
    3 = Default, based on what is available.
"""
config = '--psm 12 --oem 2 --dpi ' + str(dpi)


In [4]:
# store each pdf images from the pdfs directory
dir_images = []
# TODO(leommiranda): should I use cropbox instead of mediabox?
for pdf_path in listdir(path):
    try:
        # TODO(leommiranda): substituir este erro por um conversor de pdf normal para texto
        text = dadasd
    except:
        pdf_images = convert_from_path(pdf_path=path+pdf_path, 
                                       dpi=dpi, 
                                       first_page=first_page, 
                                       last_page=last_page, 
                                       thread_count=thread_count, 
                                       size=size)
        dir_images.append(pdf_images)

In [5]:
# TODO(leommiranda): urgently automate the comparison of extracted text and real text
"""
TODO(leommiranda): Apply pytesseract.image_to_boxes OR
    a pre-trained Text Detector such as OpenCV "EAST Text Detector" 
    to remove the document borders, keeping only the text.
    This should improve the OCR accuracy.
"""
j = 0
i = 0
for pdf_images in dir_images:
    j += 1
    for pdf_img in pdf_images:
        i += 1
        pdf_img = np.array(pdf_img.convert('RGB'))
        pdf_img = cv2.cvtColor(pdf_img, cv2.COLOR_RGB2GRAY)
        # makes adaptiveThreshold run faster and remove noise
        pdf_img = cv2.bilateralFilter(pdf_img, 17, 75, 75)
        # deals better with ilumination issues than the OTSU threshold algorithm 
            # which is the tesseract default algorithm.
        # If ilumination isn't a big deal, remeber to comment out bilateralFilter() and adaptiveThreshold()
        pdf_img = cv2.adaptiveThreshold(pdf_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
        # close operation, it seems unuseful here
        # pdf_img = cv2.morphologyEx(pdf_img, cv2.MORPH_CLOSE, (21,21))
        if (i == 1 or i == 9 or i == 12 or i == 13):
            # english document
            lang = 'eng'
        else:
            # portuguese document
            lang = 'por'
        pdf_text = pytesseract.image_to_string(Image.fromarray(pdf_img), lang=lang, config=config)
        print("\n\nPATH " + str(i) + " | " + str(j) + "\n\n\n" + pdf_text)

TesseractError: (1, "read_params_file: Can't open 450 Failed loading language 'eng' Tesseract couldn't load any languages! Could not initialize tesseract.")

In [ ]:
j = 0
i = 0
for pdf_images in dir_images:
    j += 1
    for pdf_img in pdf_images:
        i += 1
        pdf_img = np.array(pdf_img.convert('RGB'))
        pdf_img = cv2.cvtColor(pdf_img, cv2.COLOR_RGB2GRAY)
        ## if ilumination is not an issue, it's OK to comment out the bilateralFilter() and the adaptiveThreshold()
        # pdf_img = cv2.bilateralFilter(pdf_img, 17, 75, 75)
        # pdf_img = cv2.adaptiveThreshold(pdf_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
        ## close operation, it seems unuseful here
        # pdf_img = cv2.morphologyEx(pdf_img, cv2.MORPH_CLOSE, (21,21))
        if (i == 1 or i == 9 or i == 12 or i == 13):
            # english
            lang = 'eng'
        else:
            # portuguese
            lang = 'por'
        pdf_text = pytesseract.image_to_string(Image.fromarray(pdf_img), lang=lang, config=config)
        print("\n\nPATH " + str(i) + " | " + str(j) + "\n\n\n" + pdf_text)